In [2]:
from email.header import decode_header
import re
from imap_tools import MailBox
from imap_tools import AND
import pandas as pd
import dateparser
from dateparser.search import search_dates

In [161]:
password = "7PP4ZDXqnv"
username = "lazutkina@cs.vsu.ru"
host = "info.vsu.ru"

mailbox = MailBox(host)
mailbox.login(username=username, password=password, initial_folder='INBOX') 
mails = mailbox.fetch()

In [162]:
def mails_to_df(mails):
    columns = ['id', 'subject', 'from', 'text', 'date', 'attachment_filename', 'attachment_content_type', 'headers']
    mails_list = []
    for msg in mails:
        attachment_filename = ",".join(str(att.filename) for att in msg.attachments)
        attachment_content_type = ",".join(str(att.content_type) for att in msg.attachments)
        
        mails_list.append([msg.uid, msg.subject, msg.from_, msg.text, msg.date_str, attachment_filename, attachment_content_type, msg.headers])

    df = pd.DataFrame(data=mails_list)
    df.columns = columns
    return df

## Preprocessing

In [163]:
source_df = mails_to_df(mails)
source_df

,id,subject,from,text,date,attachment_filename,attachment_content_type,headers
0,70,FW: Вестник САИТ 1-2018,cs-it@cs.vsu.ru,\r\n\r\n-----Original Message-----\r\nFrom: Па...,"Thu, 16 May 2024 11:53:29 +0300",,,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
1,71,FW: Проверка статьи,cs-it@cs.vsu.ru,\r\n \r\nFrom: Михаил Горчаков <gorchakovmiha...,"Thu, 16 May 2024 11:53:32 +0300",Исправления_Горчаков.docx,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
2,72,FW: e-mail Баевой Н.Б.,cs-it@cs.vsu.ru,\r\n \r\nFrom: Аснина Н.Г. <andrey050569@yand...,"Thu, 16 May 2024 11:53:29 +0300",,,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
3,73,FW: от Матвеева,cs-it@cs.vsu.ru,\r\n \r\nFrom: Матвеев Михаил <mgmatveev@yand...,"Thu, 16 May 2024 11:53:30 +0300",Рецензия.docx,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
4,74,FW: рецензия,cs-it@cs.vsu.ru,\r\n\r\n-----Original Message-----\r\nFrom: Al...,"Thu, 16 May 2024 11:53:30 +0300",рецензия_Шмырин.docx,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
...,...,...,...,...,...,...,...,...
1132,1213,FWD:Re: Заседание редколлегии,cs-it@cs.vsu.ru,"Вероника, привет!\r\nСреда - единственный день...","Thu, 16 May 2024 12:06:45 +0300",,,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
1133,1214,FWD:Re[2]: сверка статьи,cs-it@cs.vsu.ru,"Вероника Викторовна, на сайте уже появился 2 н...","Thu, 16 May 2024 12:06:45 +0300",,,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
1134,1215,FWD:Re: статья на рецензирование,cs-it@cs.vsu.ru,"Уважаемая Вероника Викторовна,\r\nпредставленн...","Thu, 16 May 2024 12:06:45 +0300",,,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
1135,1216,FWD:,cs-it@cs.vsu.ru,Уважаемые Коллеги!\n\nПрошу Вас рассмотреть мо...,"Thu, 16 May 2024 12:06:45 +0300",4прост. фрактал 22342+.doc,application/msword,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...


In [164]:
source_df.to_csv("files/1_source_mails.csv", index=False)

In [165]:
df = source_df.copy()

In [169]:
def preprocess_date(df):
    for index, row in df.iterrows():
        headers = row['headers']
        text = row['text']
        date = row['date']
        if 'x-original-date' in headers:
            date = headers['x-original-date'][0]
            
        if 'Sent:' in text:
            date = re.search(r'Sent: (.*?)\n', text).group(1)
            
        dp = dateparser.parse(date)
        if dp is not None:
            date = dp.strftime("%d.%m.%Y %H:%M:%S")
        else:
            date = search_dates(date)[0][1].strftime("%d.%m.%Y %H:%M:%S")
            
        df.at[index, 'date'] = date
    return df

In [170]:
df = preprocess_date(df)
df.head()

,id,subject,from,text,date,attachment_filename,attachment_content_type,headers
0,70,FW: Вестник САИТ 1-2018,cs-it@cs.vsu.ru,\r\n\r\n-----Original Message-----\r\nFrom: Па...,10.06.2018 10:01:00,,,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
1,71,FW: Проверка статьи,cs-it@cs.vsu.ru,\r\n \r\nFrom: Михаил Горчаков <gorchakovmiha...,03.06.2018 23:57:00,Исправления_Горчаков.docx,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
2,72,FW: e-mail Баевой Н.Б.,cs-it@cs.vsu.ru,\r\n \r\nFrom: Аснина Н.Г. <andrey050569@yand...,11.06.2018 11:38:00,,,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
3,73,FW: от Матвеева,cs-it@cs.vsu.ru,\r\n \r\nFrom: Матвеев Михаил <mgmatveev@yand...,08.06.2018 08:49:00,Рецензия.docx,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
4,74,FW: рецензия,cs-it@cs.vsu.ru,\r\n\r\n-----Original Message-----\r\nFrom: Al...,07.06.2018 00:57:00,рецензия_Шмырин.docx,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...


In [173]:
fixed_dates_df = df.copy()

In [188]:
df = fixed_dates_df.copy()
df.head()

,id,subject,from,text,date,attachment_filename,attachment_content_type,headers
0,70,FW: Вестник САИТ 1-2018,cs-it@cs.vsu.ru,\r\n\r\n-----Original Message-----\r\nFrom: Па...,10.06.2018 10:01:00,,,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
1,71,FW: Проверка статьи,cs-it@cs.vsu.ru,\r\n \r\nFrom: Михаил Горчаков <gorchakovmiha...,03.06.2018 23:57:00,Исправления_Горчаков.docx,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
2,72,FW: e-mail Баевой Н.Б.,cs-it@cs.vsu.ru,\r\n \r\nFrom: Аснина Н.Г. <andrey050569@yand...,11.06.2018 11:38:00,,,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
3,73,FW: от Матвеева,cs-it@cs.vsu.ru,\r\n \r\nFrom: Матвеев Михаил <mgmatveev@yand...,08.06.2018 08:49:00,Рецензия.docx,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
4,74,FW: рецензия,cs-it@cs.vsu.ru,\r\n\r\n-----Original Message-----\r\nFrom: Al...,07.06.2018 00:57:00,рецензия_Шмырин.docx,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...


In [189]:
def preprocess_from(df):
    for index, row in df.iterrows():
        headers = row['headers']
        text = row['text']
        x_from = row['from']
        if 'x-original-from' in headers:
            x_from = headers['x-original-from'][0]
        elif 'From:' in text:
            x_from = re.search(r'From: (.*?)\n', text).group(1)
            
        if len(decode_header(x_from)) == 2:
            encoding = decode_header(x_from)[0][1]
            name = decode_header(x_from)[0][0]
            if encoding is not None:
                name = name.decode(encoding)
            
            address = decode_header(x_from)[1][0].decode()
            x_from = name + ' ' + address
            
        df.at[index, 'from'] = x_from
    return df

In [190]:
preprocess_from(df)

,id,subject,from,text,date,attachment_filename,attachment_content_type,headers
0,70,FW: Вестник САИТ 1-2018,Павловский Максим Викторович <pmv-160570@yande...,\r\n\r\n-----Original Message-----\r\nFrom: Па...,10.06.2018 10:01:00,,,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
1,71,FW: Проверка статьи,Михаил Горчаков <gorchakovmihail@mail.ru> \r,\r\n \r\nFrom: Михаил Горчаков <gorchakovmiha...,03.06.2018 23:57:00,Исправления_Горчаков.docx,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
2,72,FW: e-mail Баевой Н.Б.,Аснина Н.Г. <andrey050569@yandex.ru> \r,\r\n \r\nFrom: Аснина Н.Г. <andrey050569@yand...,11.06.2018 11:38:00,,,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
3,73,FW: от Матвеева,Матвеев Михаил <mgmatveev@yandex.ru> \r,\r\n \r\nFrom: Матвеев Михаил <mgmatveev@yand...,08.06.2018 08:49:00,Рецензия.docx,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
4,74,FW: рецензия,Alexander Krylovetsky <aakryl@sc.vsu.ru> \r,\r\n\r\n-----Original Message-----\r\nFrom: Al...,07.06.2018 00:57:00,рецензия_Шмырин.docx,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
...,...,...,...,...,...,...,...,...
1132,1213,FWD:Re: Заседание редколлегии,Irina Voronina <irina.voronina@gmail.com>,"Вероника, привет!\r\nСреда - единственный день...",04.09.2018 16:11:03,,,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
1133,1214,FWD:Re[2]: сверка статьи,сергеев александр <sergeev00765@mail.ru>,"Вероника Викторовна, на сайте уже появился 2 н...",05.09.2018 09:01:06,,,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
1134,1215,FWD:Re: статья на рецензирование,"""Sergey A. Zapryagaev"" <zsa@cs.vsu.ru>","Уважаемая Вероника Викторовна,\r\nпредставленн...",06.09.2018 15:35:28,,,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
1135,1216,FWD:,Александр Щербаков <a.scherbakov58@yahoo.com>,Уважаемые Коллеги!\n\nПрошу Вас рассмотреть мо...,14.04.2018 00:00:00,4прост. фрактал 22342+.doc,application/msword,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...


In [191]:
fixed_from_df = df.copy()

In [192]:
df['date'] = pd.to_datetime(df['date'], format="%d.%m.%Y %H:%M:%S")
df.head()

,id,subject,from,text,date,attachment_filename,attachment_content_type,headers
0,70,FW: Вестник САИТ 1-2018,Павловский Максим Викторович <pmv-160570@yande...,\r\n\r\n-----Original Message-----\r\nFrom: Па...,2018-06-10 10:01:00,,,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
1,71,FW: Проверка статьи,Михаил Горчаков <gorchakovmihail@mail.ru> \r,\r\n \r\nFrom: Михаил Горчаков <gorchakovmiha...,2018-06-03 23:57:00,Исправления_Горчаков.docx,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
2,72,FW: e-mail Баевой Н.Б.,Аснина Н.Г. <andrey050569@yandex.ru> \r,\r\n \r\nFrom: Аснина Н.Г. <andrey050569@yand...,2018-06-11 11:38:00,,,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
3,73,FW: от Матвеева,Матвеев Михаил <mgmatveev@yandex.ru> \r,\r\n \r\nFrom: Матвеев Михаил <mgmatveev@yand...,2018-06-08 08:49:00,Рецензия.docx,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
4,74,FW: рецензия,Alexander Krylovetsky <aakryl@sc.vsu.ru> \r,\r\n\r\n-----Original Message-----\r\nFrom: Al...,2018-06-07 00:57:00,рецензия_Шмырин.docx,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...


In [195]:
def preprocess_subject(subjects: list[str]):
    column = []
    for s in subjects:
        if s:
            s = s.removeprefix("FWD:").removeprefix("Fwd:")
            column.append(s)
        else:
            column.append('Нет темы')
    return column


In [196]:
df['subject'] = preprocess_subject(df['subject'])
df

,id,subject,from,text,date,attachment_filename,attachment_content_type,headers
11,82,Повторная отправка,Павловский Максим Викторович <pmv-160570@yand...,Добрый вечер!\r\nОтсылаю материалы рецензии на...,2016-01-14 20:47:03,Рецензия Гаршиной.doc,application/msword,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
12,83,Re: Шаблон рецензии (новый),Юрий Нечаев <nechaev_ub@mail.ru>,Спасибо.Нечаев.,2018-02-04 19:56:26,,,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
19,90,RE: статья Вестник САИТ Рецензия,Толстобров А.П. <tap@vsu.ru>,Добрый день!\r\nПосылаю рецензию\r\n\r\nС уваж...,2018-02-09 17:36:00,Рецензия Толстоброва на статью А.С. Платонова....,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
27,98,RE: статья для рецензирования,Вялых Сергей Ариевич <vyalyh@govvrn.ru>,"Доброе время суток, Вероника Викторовна!\r\n\r...",2018-02-09 17:54:00,рецензия_на_Проблема защита_пользователей.docx,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
21,92,RE: рецензия на статью,Толстобров А.П. <tap@vsu.ru>,Добрый день!\r\nПосылаю рецензию\r\n\r\nС уваж...,2018-02-11 22:37:00,Рецензия Толстоброва на статью Черненькая-Маге...,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
...,...,...,...,...,...,...,...,...
987,1065,Re: статья на рецензию,Алексей Буховец <abuhovets@mail.ru>,Высылаю рецензию на материал А.М. Сагдатуллина...,2019-11-14 09:40:50,Рецензии Сагдатуллина А_М_САИТ.docx,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
990,1068,статья,hein minzaw <heinminnzaw13@gmail.com>,Здравствуйте !\r\nМою отправлен нучную статью ...,2019-11-14 14:40:59,"Рис1.docx,Рис2.docx,Статья .doc",application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
991,1069,Re: Королев= ответы на замечания рецензента,Сергей Медведев <s_n_medvedev@mail.ru>,"Здравствуйте.\nСчитаю, что статья может быть о...",2019-11-15 00:03:13,"Ответ после повторного рецензирования.pdf,Реце...","application/pdf,application/vnd.openxmlformats...",{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
988,1066,Re: Конденсированные среды,CONDENSED MATTER AND INTERPHASES <kcmf@main.vs...,"Добрый день, Виктория Викторовна!\r\n\r\nЮрист...",2019-11-15 09:43:16,,,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...


In [ ]:
df.to_csv('files/2_fixed_values.csv', index=False)

In [193]:
df = df.sort_values(by='date')
df.head()

,id,subject,from,text,date,attachment_filename,attachment_content_type,headers
11,82,FWD:Повторная отправка,Павловский Максим Викторович <pmv-160570@yand...,Добрый вечер!\r\nОтсылаю материалы рецензии на...,2016-01-14 20:47:03,Рецензия Гаршиной.doc,application/msword,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
12,83,FWD:Re: Шаблон рецензии (новый),Юрий Нечаев <nechaev_ub@mail.ru>,Спасибо.Нечаев.,2018-02-04 19:56:26,,,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
19,90,FWD:RE: статья Вестник САИТ Рецензия,Толстобров А.П. <tap@vsu.ru>,Добрый день!\r\nПосылаю рецензию\r\n\r\nС уваж...,2018-02-09 17:36:00,Рецензия Толстоброва на статью А.С. Платонова....,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
27,98,FWD:RE: статья для рецензирования,Вялых Сергей Ариевич <vyalyh@govvrn.ru>,"Доброе время суток, Вероника Викторовна!\r\n\r...",2018-02-09 17:54:00,рецензия_на_Проблема защита_пользователей.docx,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
21,92,FWD:RE: рецензия на статью,Толстобров А.П. <tap@vsu.ru>,Добрый день!\r\nПосылаю рецензию\r\n\r\nС уваж...,2018-02-11 22:37:00,Рецензия Толстоброва на статью Черненькая-Маге...,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...


In [194]:
df.to_csv('files/3_sorted_df.csv', index=False)

In [197]:
fixed_subject_df = df.copy()

In [200]:
df.shape

(1137, 8)

In [214]:
df = df.drop_duplicates('text')

In [215]:
df.shape

(825, 8)

In [216]:
df.to_csv('files/4_drop_duplicates_df.csv', index=False)

## Clean text

In [222]:
import string

def preprocess_text(texts):
    column = []
    for t in texts:
        t = re.sub(r'\n+', ' ', t)
        t = re.sub(r'\r+', ' ', t)
        t = re.sub(r'\t+', ' ', t)
        # t = re.sub("[" + string.punctuation + "]", " ", t)
        t = re.sub(r'-{2,}', '', t)
        t = re.sub(r'\s+', ' ', t)
        column.append(t)
    return column

In [223]:
df['text'] = preprocess_text(df['text'])
df.head()

,id,subject,from,text,date,attachment_filename,attachment_content_type,headers
11,82,Повторная отправка,Павловский Максим Викторович <pmv-160570@yand...,Добрый вечер Отсылаю материалы рецензии на раб...,2016-01-14 20:47:03,Рецензия Гаршиной.doc,application/msword,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
12,83,Re: Шаблон рецензии (новый),Юрий Нечаев <nechaev_ub@mail.ru>,Спасибо Нечаев,2018-02-04 19:56:26,,,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
19,90,RE: статья Вестник САИТ Рецензия,Толстобров А.П. <tap@vsu.ru>,Добрый день Посылаю рецензию С уважением Толст...,2018-02-09 17:36:00,Рецензия Толстоброва на статью А.С. Платонова....,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
27,98,RE: статья для рецензирования,Вялых Сергей Ариевич <vyalyh@govvrn.ru>,Доброе время суток Вероника Викторовна Высылаю...,2018-02-09 17:54:00,рецензия_на_Проблема защита_пользователей.docx,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
21,92,RE: рецензия на статью,Толстобров А.П. <tap@vsu.ru>,Добрый день Посылаю рецензию С уважением Толст...,2018-02-11 22:37:00,Рецензия Толстоброва на статью Черненькая-Маге...,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...


In [225]:
df['text'].iloc[100]

'Здравствуйте Пожалуйста рассмотрите возможность публикации моей статьи в Вашем журнале С уважением Леонид Эрштейн'

In [226]:
df['text'].to_csv('files/5_texts.csv')

In [227]:
df['text']

11     Добрый вечер Отсылаю материалы рецензии на раб...
12                                       Спасибо Нечаев 
19     Добрый день Посылаю рецензию С уважением Толст...
27     Доброе время суток Вероника Викторовна Высылаю...
21     Добрый день Посылаю рецензию С уважением Толст...
                             ...                        
987    Высылаю рецензию на материал А М Сагдатуллина ...
990    Здравствуйте Мою отправлен нучную статью с наз...
991    Здравствуйте Считаю что статья может быть опуб...
988    Добрый день Виктория Викторовна Юристы очень с...
582    Привет друг Это Sasa из Тяньцзиня Semri Bearin...
Name: text, Length: 825, dtype: object

## предобработка текста для кластеризации

In [264]:
texts_df = df[['id', 'text', 'subject']]

In [265]:
def clean_text(texts):
    column = []
    for s in texts:
        s = s.lower()
        s = s.removeprefix('re')
        s = s.replace('fwd', '')
        s = s.replace('fw', '')
        s = re.sub(r'[0-9]+', '', s)
        s = re.sub("[" + string.punctuation + "]", " ", s)
        s = re.sub(r'\s+', ' ', s)
        column.append(s)
    return column

In [266]:
texts_df['text'] = clean_text(texts_df['text'])
texts_df['text'].head()

C:\Users\08642\AppData\Local\Temp\ipykernel_22400\3963136588.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts_df['text'] = clean_text(texts_df['text'])


11    добрый вечер отсылаю материалы рецензии на раб...
12                                      спасибо нечаев 
19    добрый день посылаю рецензию с уважением толст...
27    доброе время суток вероника викторовна высылаю...
21    добрый день посылаю рецензию с уважением толст...
Name: text, dtype: object

In [267]:
texts_df['subject'] = clean_text(texts_df['subject'])

C:\Users\08642\AppData\Local\Temp\ipykernel_22400\3295035228.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts_df['subject'] = clean_text(texts_df['subject'])


In [268]:
texts_df['category'] = ''

C:\Users\08642\AppData\Local\Temp\ipykernel_22400\1287844357.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts_df['category'] = ''


In [269]:
texts_df.head()

,id,text,subject,category
11,82,добрый вечер отсылаю материалы рецензии на раб...,повторная отправка,
12,83,спасибо нечаев,шаблон рецензии новый,
19,90,добрый день посылаю рецензию с уважением толст...,статья вестник саит рецензия,
27,98,доброе время суток вероника викторовна высылаю...,статья для рецензирования,
21,92,добрый день посылаю рецензию с уважением толст...,рецензия на статью,


In [270]:
texts_df.to_csv("files/texts_to_categorization.csv", index=False)

### Разметка

In [ ]:
categories = ['submitting an article', 'checking an article', 'revision', 'ready for review', 'reviewing', 'rejected', 'publication']
categories_rus = ['подача статьи', 'проверка статьи', 'доработка', 'готово к рецензированию', 'рецензирование', 'отклонена', 'публикация']

In [ ]:
submitting_keywords = 'статья в вестник вгу, статья для, статья для публикации, статья на публикацию, публикация статьи, статья, публикация, статья вак'
ready_for_review_keywords = 'статья для рецензирования, статья экспертное заключение'

In [ ]:
# def fill_category(dataframe: pd.DataFrame):
#     col = []
#     for i, row in dataframe.iterrows():
        

In [228]:
new_df = pd.read_csv('files/drop_duplicates_df.csv')
new_df

,id,subject,from,text,date,attachment_filename,attachment_content_type,headers
0,82,Повторная отправка,Павловский Максим Викторович <pmv-160570@yand...,Добрый вечер!\r\nОтсылаю материалы рецензии на...,2016-01-14 20:47:03,Рецензия Гаршиной.doc,application/msword,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
1,83,Re: Шаблон рецензии (новый),Юрий Нечаев <nechaev_ub@mail.ru>,Спасибо.Нечаев.,2018-02-04 19:56:26,NaN,NaN,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
2,90,RE: статья Вестник САИТ Рецензия,Толстобров А.П. <tap@vsu.ru>,Добрый день!\r\nПосылаю рецензию\r\n\r\nС уваж...,2018-02-09 17:36:00,Рецензия Толстоброва на статью А.С. Платонова....,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
3,98,RE: статья для рецензирования,Вялых Сергей Ариевич <vyalyh@govvrn.ru>,"Доброе время суток, Вероника Викторовна!\r\n\r...",2018-02-09 17:54:00,рецензия_на_Проблема защита_пользователей.docx,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
4,92,RE: рецензия на статью,Толстобров А.П. <tap@vsu.ru>,Добрый день!\r\nПосылаю рецензию\r\n\r\nС уваж...,2018-02-11 22:37:00,Рецензия Толстоброва на статью Черненькая-Маге...,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
...,...,...,...,...,...,...,...,...
820,1065,Re: статья на рецензию,Алексей Буховец <abuhovets@mail.ru>,Высылаю рецензию на материал А.М. Сагдатуллина...,2019-11-14 09:40:50,Рецензии Сагдатуллина А_М_САИТ.docx,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
821,1068,статья,hein minzaw <heinminnzaw13@gmail.com>,Здравствуйте !\r\nМою отправлен нучную статью ...,2019-11-14 14:40:59,"Рис1.docx,Рис2.docx,Статья .doc",application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
822,1069,Re: Королев= ответы на замечания рецензента,Сергей Медведев <s_n_medvedev@mail.ru>,"Здравствуйте.\nСчитаю, что статья может быть о...",2019-11-15 00:03:13,"Ответ после повторного рецензирования.pdf,Реце...","application/pdf,application/vnd.openxmlformats...",{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
823,1066,Re: Конденсированные среды,CONDENSED MATTER AND INTERPHASES <kcmf@main.vs...,"Добрый день, Виктория Викторовна!\r\n\r\nЮрист...",2019-11-15 09:43:16,NaN,NaN,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...


In [229]:
df['subject'].to_csv('files/subjects.csv', index=False)

In [242]:
subjects = df[['id', 'subject']]
subjects

,id,subject
11,82,Повторная отправка
12,83,Re: Шаблон рецензии (новый)
19,90,RE: статья Вестник САИТ Рецензия
27,98,RE: статья для рецензирования
21,92,RE: рецензия на статью
...,...,...
987,1065,Re: статья на рецензию
990,1068,статья
991,1069,Re: Королев= ответы на замечания рецензента
988,1066,Re: Конденсированные среды


In [243]:
subjects['subject'] = clean_text(subjects['subject'])

C:\Users\08642\AppData\Local\Temp\ipykernel_22400\4018698362.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subjects['subject'] = clean_subject(subjects['subject'])


In [246]:
uniq_sbj = subjects.drop_duplicates(subset='subject')
uniq_sbj

,id,subject
11,82,повторная отправка
12,83,шаблон рецензии новый
19,90,статья вестник саит рецензия
27,98,статья для рецензирования
21,92,рецензия на статью
...,...,...
1000,1079,сведения об авторах doc статья кротов docx экс...
986,1064,статья колегова о а
987,1065,статья на рецензию
991,1069,королев ответы на замечания рецензента


In [247]:
uniq_sbj.to_csv('files/unique_subjects.csv', index=False)

## navec embeddings

In [249]:
import pandas as pd
import numpy as np
import torchtext
import torch
from torchtext.vocab import build_vocab_from_iterator
from collections import Counter
import natasha
from natasha import Segmenter, Doc
from torch.utils.data import TensorDataset, random_split, DataLoader
import torch
from torch import nn


import torchmetrics
import lightning.pytorch as pl
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks import ModelSummary
from lightning.pytorch.tuner import Tuner
from torch.optim.lr_scheduler import ReduceLROnPlateau
from lightning.pytorch.callbacks import Callback
import matplotlib.pyplot as plt

from navec import Navec
from slovnet.model.emb import NavecEmbedding

F:\Projects\Python\techAI\MailManagementApp\lib\site-packages\torchtext\vocab\__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
F:\Projects\Python\techAI\MailManagementApp\lib\site-packages\torchtext\utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


ModuleNotFoundError: No module named 'torchmetrics'

In [252]:
path = '../navec_hudlit_v1_12B_500K_300d_100q.tar'
navec = Navec.load(path)

## Ярги парсер

In [76]:
test_df = pd.read_csv('files/drop_duplicates_df.csv')
test_df.head() 

,id,subject,from,text,date,attachment_filename,attachment_content_type,headers
0,82,Повторная отправка,Павловский Максим Викторович <pmv-160570@yand...,Добрый вечер!\r\nОтсылаю материалы рецензии на...,2016-01-14 20:47:03,Рецензия Гаршиной.doc,application/msword,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
1,83,Re: Шаблон рецензии (новый),Юрий Нечаев <nechaev_ub@mail.ru>,Спасибо.Нечаев.,2018-02-04 19:56:26,NaN,NaN,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
2,90,RE: статья Вестник САИТ Рецензия,Толстобров А.П. <tap@vsu.ru>,Добрый день!\r\nПосылаю рецензию\r\n\r\nС уваж...,2018-02-09 17:36:00,Рецензия Толстоброва на статью А.С. Платонова....,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
3,98,RE: статья для рецензирования,Вялых Сергей Ариевич <vyalyh@govvrn.ru>,"Доброе время суток, Вероника Викторовна!\r\n\r...",2018-02-09 17:54:00,рецензия_на_Проблема защита_пользователей.docx,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
4,92,RE: рецензия на статью,Толстобров А.П. <tap@vsu.ru>,Добрый день!\r\nПосылаю рецензию\r\n\r\nС уваж...,2018-02-11 22:37:00,Рецензия Толстоброва на статью Черненькая-Маге...,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...


In [36]:
text = test_df.iloc[20]['text']
text

'Здравствуйте!\r\n\r\nПишет Вам аспирант математического факультета ВГУ Литвинов Дмитрий!\r\nЯ собираюсь отправить свою статью в журнал "Вестник ВГУ. Серия: Системный анализ и информационные технологии". В связи с этим у меня есть несколько вопросов:\r\n\r\n\r\n1)Принимаете ли Вы статьи аспирантов?\r\n2)Какие документы и прочие материалы я должен предоставить вместе со статьей в редакцию?\r\n3)Можно ли предоставить электронную версию статьи в формате tex?\r\n4)Как долго происходит процесс рецензирования в журнале?\r\n\r\nЗаранее спасибо!\r\n'

In [32]:
from yargy.tokenizer import MorphTokenizer
from yargy.predicates import gram, dictionary, type as tp
from yargy import Parser, rule, and_, not_, or_
from yargy.predicates import gram,  eq, normalized
from yargy.pipelines import morph_pipeline
from yargy.interpretation import fact
from yargy.tokenizer import RULES
from ipymarkup import show_span_box_markup as show_markup

In [48]:
TOKENIZER = MorphTokenizer()
list(TOKENIZER('поинтересуюсь'))

[MorphToken(
     value='поинтересуюсь',
     span=[0, 13),
     type='RU',
     forms=[Form('поинтересоваться', Grams(1per,VERB,futr,indc,intr,perf,sing))]
 )]

In [82]:
subjects = test_df[['id', 'subject', 'attachment_filename']]
subjects

,id,subject,attachment_filename
0,82,Повторная отправка,Рецензия Гаршиной.doc
1,83,Re: Шаблон рецензии (новый),NaN
2,90,RE: статья Вестник САИТ Рецензия,Рецензия Толстоброва на статью А.С. Платонова....
3,98,RE: статья для рецензирования,рецензия_на_Проблема защита_пользователей.docx
4,92,RE: рецензия на статью,Рецензия Толстоброва на статью Черненькая-Маге...
...,...,...,...
820,1065,Re: статья на рецензию,Рецензии Сагдатуллина А_М_САИТ.docx
821,1068,статья,"Рис1.docx,Рис2.docx,Статья .doc"
822,1069,Re: Королев= ответы на замечания рецензента,"Ответ после повторного рецензирования.pdf,Реце..."
823,1066,Re: Конденсированные среды,NaN


In [83]:
subjects['category'] = ''


C:\Users\08642\AppData\Local\Temp\ipykernel_10472\2351832943.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subjects['category'] = ''


In [117]:
Category = fact('Category', ['name'])
article = morph_pipeline(
    ['опубликовать','публикация','опубликования','разместить',
        'материалы','размещение','печатать','напечатать','принять']
).interpretation(Category.name.const('подача'))

review = morph_pipeline(
    ['рецензия на статью','рецензия','отрецензировать',
        'рецензирование','рецензент']
).interpretation(Category.name.const('рецензия'))

question = morph_pipeline(
    ['вопрос','подсказать','сказать','?','узнать',
        'интересоваться','поинтересоваться', 'интересует','повод']
).interpretation(Category.name.const('вопрос'))

ARTICLE = rule(
    or_(article, review, question)).interpretation(Category)
parser = Parser(ARTICLE)
# matches = list(parser.findall(text))
# spans = [_.span for _ in matches]
# show_markup(text, spans)
# for match in parser.findall(text):
#     print (match.fact)

In [118]:
for index, row in subjects.iterrows():
    tx = row['subject']
    if (tx is not None) or (tx != 'NaN') or (str(tx) != 'nan'):
        for match in parser.findall(str(tx)):
                category = match.fact
                if category is not None:
                    subjects.at[index, 'category'] = str(category.name)
                else:
                    subjects.at[index, 'category'] = 'Nan'
    else:
        subjects.at[index, 'category'] = 'Nan'


In [119]:
subjects[subjects['id'] == 143]

,id,subject,attachment_filename,category
62,143,Вопрос по публикации в вашем журнале,NaN,подача


In [115]:
submitting = subjects.loc[subjects['category'].isin(['подача'])]
submitting

,id,subject,attachment_filename,category
5,80,Re: сверка статьи,NaN,подача
6,79,Re: сверка статьи,NaN,подача
7,81,Re: сверка статьи,NaN,подача
8,123,статья_Шубина_Васева,"Авторы_свед.docx,Рисунки_Шубина_Васева.rar,Ста...",подача
9,99,Статьи в №1 2018 года,"Статья 1 ( Нечаев и соавторы) Текст.doc,Ста...",подача
...,...,...,...,...
809,1053,Re: Запрос на публикацию.,NaN,подача
817,1078,"Статья Кротов.docx, экспертное заключение 11-2...","Статья Кротов.docx,экспертное заключение 11-2....",подача
818,1079,"Сведения об авторах.doc, Статья Кротов.docx, э...","Сведения об авторах.doc,Статья Кротов.docx,экс...",подача
819,1064,Re[2]: Статья Колегова О.А.,NaN,подача


In [116]:
submitting.to_csv('files/submitting.csv', index=False)

In [17]:
from yargy import Parser, rule
from yargy.predicates import gram, dictionary
from yargy.tokenizer import EMAIL_RULE, PHONE_RULE
from yargy.tokenizer import Tokenizer

tokenizer = Tokenizer().remove_types('EOL').add_rules(EMAIL_RULE, PHONE_RULE)
# list(tokenizer(text))
parser = Parser(tokenizer)
tokens = tokenizer(text)
parser.match(tokens);

AttributeError: 'Tokenizer' object has no attribute 'activate'

Разделение сообщения

In [41]:
text = test_df.loc[test_df['id']=='115']['text']
text

Series([], Name: text, dtype: object)

In [ ]:
delimiters = ['-----Original Message-----\n', '-------- Пересылаемое сообщение --------\n', 'пишет:\n', ]